In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd/content/drive/MyDrive/Colab Notebooks

/content/drive/MyDrive/Colab Notebooks


In [4]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS = 10


In [5]:
# Load the labels from the Excel sheet
labels_df = pd.read_csv('./miml_dataset/miml_labels_2.csv')
labels_df.head()


,Filenames,labels
0,1.jpg,desert
1,2.jpg,desert
2,3.jpg,desert
3,4.jpg,"desert,mountains"
4,5.jpg,desert


In [6]:
print(labels_df)


     Filenames            labels
0        1.jpg            desert
1        2.jpg            desert
2        3.jpg            desert
3        4.jpg  desert,mountains
4        5.jpg            desert
...        ...               ...
1995  1996.jpg             trees
1996  1997.jpg             trees
1997  1998.jpg             trees
1998  1999.jpg             trees
1999  2000.jpg             trees

[2000 rows x 2 columns]


In [7]:
print(labels_df.columns)


Index(['Filenames', 'labels'], dtype='object')


In [8]:

# Check the column names in the DataFrame
print(labels_df.columns)

Index(['Filenames', 'labels'], dtype='object')


In [9]:
# Define the ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2)

In [10]:
num_classes = len(labels_df['labels'].unique())

In [11]:
train_set = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory='./miml_dataset/images',
    x_col='Filenames',
    y_col='labels',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42,
    validate_filenames=True,
    interpolation='bilinear',
    color_mode='rgb',
    save_format='jpg',
    num_classes=num_classes)

Found 1600 validated image filenames belonging to 20 classes.


In [12]:
# Load the validation set
val_set = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory='./miml_dataset/images',
    x_col='Filenames',
    y_col='labels',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation')

Found 400 validated image filenames belonging to 20 classes.


In [13]:
num_classes = len(labels_df['labels'].unique())


In [14]:
# Define the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 [==============================] - 1s 0us/step


In [15]:
# Compile and train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_set, validation_data=val_set, epochs=EPOCHS)

Epoch 1/10
50/50 [==============================] - 1476s 29s/step - loss: 1.5704 - accuracy: 0.5437 - val_loss: 51.9623 - val_accuracy: 0.0000e+00
Epoch 2/10
50/50 [==============================] - 1320s 26s/step - loss: 1.0104 - accuracy: 0.6681 - val_loss: 4.6320 - val_accuracy: 0.0000e+00
Epoch 3/10
48/50 [===========================>..] - ETA: 49s - loss: 1.0131 - accuracy: 0.6673 

In [ ]:
model.save("resnet.h5")

In [ ]:
model

In [ ]:
# Generate predictions on the test set
test_set = train_datagen.flow_from_dataframe(
    dataframe=labels_df,
    directory='dataset',
    x_col='Filenames',
    y_col='labels',
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=1,
    shuffle=False,
    class_mode='categorical')
y_pred = model.predict(test_set)

In [ ]:
# Convert the predictions to binary labels
y_pred_bin = np.round(y_pred)

In [ ]:
# Print the classification report
print(classification_report(test_set.labels, y_pred_bin))